In [12]:
import pandas as pd
import requests

In [14]:
df = pd.read_csv('results-gpt4o-mini.csv')
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [15]:
df = df.iloc[:300]

### Q1) multi-qa-mpnet-base-dot-v1

In [8]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\aksha\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aksha\.cache\huggingface\hub\models--sentence-transformers--multi-qa-mpnet-base-dot-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
answer_llm = df.iloc[0].answer_llm
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [17]:
answer_llm_dv = embedding_model.encode(answer_llm)
answer_llm_dv

array([-4.22446728e-01, -2.24856183e-01, -3.24058503e-01, -2.84758508e-01,
        7.25647062e-03,  1.01186447e-01,  1.03716813e-01, -1.89983472e-01,
       -2.80597620e-02,  2.71588862e-01, -1.15337394e-01,  1.14666045e-01,
       -8.49585682e-02,  3.32365334e-01,  5.52722961e-02, -2.22195715e-01,
       -1.42540902e-01,  1.02519214e-01, -1.52333900e-01, -2.02912524e-01,
        1.98423788e-02,  8.38149711e-02, -5.68632245e-01,  2.32842118e-02,
       -1.67292610e-01, -2.39256859e-01, -8.05464238e-02,  2.57083215e-02,
       -8.15464184e-02, -7.39289448e-02, -2.61549890e-01,  1.92575678e-02,
        3.22909355e-01,  1.90357119e-01, -9.34726340e-05, -2.13165775e-01,
        2.88942344e-02, -1.79528613e-02, -5.92758060e-02,  1.99918389e-01,
       -4.75171208e-02,  1.71634033e-01, -2.45916974e-02, -9.38061550e-02,
       -3.57002735e-01,  1.33263648e-01,  1.94045693e-01, -1.18530534e-01,
        4.56915081e-01,  1.47728130e-01,  3.35945159e-01, -1.86959460e-01,
        2.45955005e-01, -

#### -0.42

### Q2)

In [42]:
from tqdm.auto import tqdm

results_gpt4o = df.to_dict(orient='records')
results_gpt4o[0]  

{'answer_llm': 'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).',
 'answer_orig': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'document': '0227b872',
 'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp'}

In [44]:
def compute_dot_product(record):
    llm_en = embedding_model.encode(record['answer_llm'])
    orig_en = embedding_model.encode(record['answer_orig'])
    return llm_en.dot(orig_en)

In [45]:
similarity = []

for record in tqdm(results_gpt4o):
    sim = compute_dot_product(record)
    similarity.append(sim)

  0%|          | 0/300 [00:00<?, ?it/s]

In [47]:
import numpy as np
percentil_75 = np.percentile(similarity, 75)
percentil_75

31.674312114715576

### 31.67

### Q3)

In [48]:
def compute_cosine_noemalize(record):
    llm_en = embedding_model.encode(record['answer_llm'])
    orig_en = embedding_model.encode(record['answer_orig'])
    
    # Calculate the norm for both vectors
    llm_norm = np.sqrt((llm_en * llm_en).sum())
    orig_norm = np.sqrt((orig_en * orig_en).sum())
    
    # Normalize the vectors
    llm_en_norm = llm_en / llm_norm
    orig_en_norm = orig_en / orig_norm
    
    # Compute the dot product of the normalized vectors
    return np.dot(llm_en_norm, orig_en_norm)

In [49]:
similarity_norm = []

for record in tqdm(results_gpt4o):
    sim = compute_cosine_noemalize(record)
    similarity_norm.append(sim)

  0%|          | 0/300 [00:00<?, ?it/s]

In [53]:
percentil_75_norm = np.percentile(similarity_norm, 75)
percentil_75_norm

0.8362348973751068

### 0.83

### Q4. Rouge

In [55]:
!pip install rouge

In [58]:
from rouge import Rouge

In [62]:

rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(df['answer_llm'], df['answer_orig'])[10]

In [63]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

### 0.45

### Q)5. Average rouge score

In [71]:
f_values = [scores['f'] for scores in scores.values()]
f_values

[0.45454544954545456, 0.21621621121621637, 0.393939388939394]

In [72]:
np.average(f_values)

0.35490034990035496

### 0.35

### Q6) Average rouge score for all the data points

In [74]:
rouge_scores = []
for i in tqdm(range(len(df))):
    scores = rouge_scorer.get_scores(df['answer_llm'], df['answer_orig'])[i]
    rouge_scores.append(scores)

  0%|          | 0/300 [00:00<?, ?it/s]

In [76]:
rouge_scores

[{'rouge-1': {'r': 0.061224489795918366,
   'p': 0.21428571428571427,
   'f': 0.09523809178130524},
  'rouge-2': {'r': 0.017543859649122806,
   'p': 0.07142857142857142,
   'f': 0.028169010918468917},
  'rouge-l': {'r': 0.061224489795918366,
   'p': 0.21428571428571427,
   'f': 0.09523809178130524}},
 {'rouge-1': {'r': 0.08163265306122448,
   'p': 0.26666666666666666,
   'f': 0.12499999641113292},
  'rouge-2': {'r': 0.03508771929824561,
   'p': 0.13333333333333333,
   'f': 0.05555555225694465},
  'rouge-l': {'r': 0.061224489795918366, 'p': 0.2, 'f': 0.09374999641113295}},
 {'rouge-1': {'r': 0.32653061224489793,
   'p': 0.5714285714285714,
   'f': 0.41558441095631643},
  'rouge-2': {'r': 0.14035087719298245,
   'p': 0.24242424242424243,
   'f': 0.17777777313333343},
  'rouge-l': {'r': 0.30612244897959184,
   'p': 0.5357142857142857,
   'f': 0.3896103849822905}},
 {'rouge-1': {'r': 0.16326530612244897, 'p': 0.32, 'f': 0.2162162117421476},
  'rouge-2': {'r': 0.03508771929824561,
   'p': 0

In [89]:
rouge_f2 = []

for i in tqdm(range(len(df))):
    for i, v in rouge_scores[i].items():
        if(i == "rouge-2"):
           rouge_f2.append(v['f'])

  0%|          | 0/300 [00:00<?, ?it/s]

In [90]:
np.average(rouge_f2)

0.20696501983423318

### 0.20